<a href="https://colab.research.google.com/github/bonitr02/datasci_6_anova/blob/main/data_sci_6_anova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HHA 507 Week 6 - ANOVA Analysis Part II

## Import Packages

In [40]:
!pip install ucimlrepo
import pandas as pd
from ucimlrepo import fetch_ucirepo
from scipy.stats import shapiro
import scipy.stats as stats


import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.formula.api import ols

## Load Dataset

In [20]:
# fetch dataset
diabetes = fetch_ucirepo(id=296)

/usr/local/lib/python3.10/dist-packages/ucimlrepo/fetch.py:97: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(data_url)


In [21]:
# data (as pandas dataframes)
X = diabetes.data.features
y = diabetes.data.targets

In [35]:
df = pd.DataFrame(X)
len(df)
#101766 values

101766

In [23]:
df.columns

Index(['race', 'gender', 'age', 'weight', 'admission_type_id',
       'discharge_disposition_id', 'admission_source_id', 'time_in_hospital',
       'payer_code', 'medical_specialty', 'num_lab_procedures',
       'num_procedures', 'num_medications', 'number_outpatient',
       'number_emergency', 'number_inpatient', 'diag_1', 'diag_2', 'diag_3',
       'number_diagnoses', 'max_glu_serum', 'A1Cresult', 'metformin',
       'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride',
       'acetohexamide', 'glipizide', 'glyburide', 'tolbutamide',
       'pioglitazone', 'rosiglitazone', 'acarbose', 'miglitol', 'troglitazone',
       'tolazamide', 'examide', 'citoglipton', 'insulin',
       'glyburide-metformin', 'glipizide-metformin',
       'glimepiride-pioglitazone', 'metformin-rosiglitazone',
       'metformin-pioglitazone', 'change', 'diabetesMed'],
      dtype='object')

In [24]:
df.dtypes

race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride                 object
acetohexamide               object
glipizide           

In [ ]:
# Will use num_medications as dependent variable and race as indepent variable with multiple levels
# Both variables are the appropriate data types

### Look for missingness

In [36]:
newdf = df[['num_medications', 'race']]
print(newdf)
len(newdf)
#101766 rows

        num_medications             race
0                     1        Caucasian
1                    18        Caucasian
2                    13  AfricanAmerican
3                    16        Caucasian
4                     8        Caucasian
...                 ...              ...
101761               16  AfricanAmerican
101762               18  AfricanAmerican
101763                9        Caucasian
101764               21        Caucasian
101765                3        Caucasian

[101766 rows x 2 columns]


101766

In [39]:
## keep only complete rows
newdf = newdf.dropna()
len(newdf)
#99493 remaining rows

<bound method NDFrame.describe of         num_medications             race
0                     1        Caucasian
1                    18        Caucasian
2                    13  AfricanAmerican
3                    16        Caucasian
4                     8        Caucasian
...                 ...              ...
101761               16  AfricanAmerican
101762               18  AfricanAmerican
101763                9        Caucasian
101764               21        Caucasian
101765                3        Caucasian

[99493 rows x 2 columns]>

## Assumption Checks

### Normality

In [ ]:
stat, p = shapiro(newdf['num_medications'])
if p > 0.05:
    print("Number of medication data follows a normal distribution. P-value is: ",p)
else:
    print("Number of medication data does not follow a normal distribution. P-value is: ",p)

In [ ]:
# Histogram
plt.hist(newdf['num_medications'], bins=20, edgecolor='k', alpha=0.7)
plt.title('Histogram of Numbers of Medications')
plt.xlabel('number of medications')
plt.ylabel('number of patients')
plt.show()

In [ ]:
groups = newdf.groupby(['race'])

for (race_status), group_df in groups:
    _, p_value = stats.shapiro(group_df['num_medications'])

    print(f"Group ({race_status}):")
    print(f"P-value from Shapiro-Wilk Test: {p_value}\n")

# greater than 0.05 is normal distribution

#### Interpretation

Num_Medications: p-value is 0.0, indicating a statistically significant difference in the distribution of the data. Visualizing the histogram confirms that the data is skewed to the left.

Race: p-value is less than 0.05 for all racial categories, indicating a statistically signficant difference in the distribution of data across racial categories

### Homoscedasticity (Equal Variances)

In [57]:
newdf['race'].value_counts()

Caucasian          76099
AfricanAmerican    19210
Hispanic            2037
Other               1506
Asian                641
Name: race, dtype: int64

In [58]:
# check homogeneous variance with Levene's test
print(stats.levene(newdf[newdf["race"] == "Caucasian"]["num_medications"],
                     newdf[newdf["race"] == "AfricanAmerican"]["num_medications"],
                   newdf[newdf["race"] == "Hispanic"]["num_medications"],
                   newdf[newdf["race"] == "Other"]["num_medications"],
                   newdf[newdf["race"] == "Asian"]["num_medications"]))

LeveneResult(statistic=9.964541877327243, pvalue=4.6475787972404207e-08)


#### Interpretation

Interpretation of the results:

The p-value is less than 0.05, indicating that there is strong evidence to suggest that the homogeneity of variance is violated.


## One-way ANOVA test

#### Interpretation

## Post-hoc Tukey Test

#### Interpretation

In [52]:
!pip install faker
from faker import Faker
import pandas as pd
from scipy import stats

fake = Faker()

# Generate a fake healthcare dataset with two groups
num_samples_per_group = 50

data = {
    "Group": ["Control"] * num_samples_per_group + ["Treatment"] * num_samples_per_group,
    "Blood_Pressure": [fake.random_int(100, 140) for _ in range(num_samples_per_group)] +
                      [fake.random_int(110, 150) for _ in range(num_samples_per_group)],
    "Cholesterol": [fake.random_int(150, 250) for _ in range(num_samples_per_group)] +
                   [fake.random_int(160, 280) for _ in range(num_samples_per_group)]
}

In [54]:
df2 = pd.DataFrame(data)
print (df2)

        Group  Blood_Pressure  Cholesterol
0     Control             112          230
1     Control             120          222
2     Control             126          158
3     Control             103          172
4     Control             132          232
..        ...             ...          ...
95  Treatment             136          275
96  Treatment             113          168
97  Treatment             138          205
98  Treatment             143          162
99  Treatment             134          204

[100 rows x 3 columns]


In [ ]:
# Convert the columns to numeric
df["Blood_Pressure"] = pd.to_numeric(df["Blood_Pressure"])
df["Cholesterol"] = pd.to_numeric(df["Cholesterol"])

# check homogeneous variance with Levene's test
print(stats.levene(df[df["Group"] == "Control"]["Blood_Pressure"],
                     df[df["Group"] == "Treatment"]["Blood_Pressure"]))
print(stats.levene(df[df["Group"] == "Control"]["Cholesterol"],
                        df[df["Group"] == "Treatment"]["Cholesterol"]))
